In [ ]:
!pip install transformers

In [ ]:
!pip install lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283835 sha256=10a2732b5a308c2cbbf642b6a2a2937cca2b7748565f9fbde4c9558784436bb6
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


In [ ]:
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 533.5/533.5 kB 3.8 MB/s eta 0:00:00


In [1]:
!pip install datasets evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.1 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
def load_dataset_func(name):
  return load_dataset(name)
imdb = load_dataset_func("imdb")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
from transformers import AutoTokenizer
def load_tokenizer_func(name):
  return AutoTokenizer.from_pretrained(name)
tokenizer = load_tokenizer_func("microsoft/deberta-base")

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [4]:
#funkcija za preprocesiranje
def preprocess_func(examples):
  return tokenizer(examples["text"],truncation = True, padding= "max_length")

In [5]:
tokenized_imdb = imdb.map(preprocess_func, batched = True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:
#prilagodjavanje dataseta za rad sa PyTorch-em
tokenized_imdb = tokenized_imdb.remove_columns(["text"])
tokenized_imdb = tokenized_imdb.rename_column("label","labels")
tokenized_imdb.set_format("torch")

In [7]:
#fino podesavanje modela se vrsi sa podskupom seta podataka, radi efikasnosti prilikom treniranja i ubrzanja samog procesa
def partial_dataset_func(train_or_test, num_of_instances):
  return tokenized_imdb[train_or_test].shuffle(seed = 42).select(range(num_of_instances))
partial_train_dataset = partial_dataset_func("train", 12500)
partial_eval_dataset = partial_dataset_func("test", 2500)

In [8]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(partial_train_dataset,shuffle = True, batch_size = 4)
eval_dataloader = DataLoader(partial_eval_dataset, batch_size = 4)

In [9]:
#odabir modela koji ce se koristiti kao klasifikator
from transformers import T5ForSequenceClassification, DebertaForSequenceClassification, BloomForSequenceClassification
model_t5_small = T5ForSequenceClassification.from_pretrained("t5-small", num_labels = 2)
model_deberta = DebertaForSequenceClassification.from_pretrained("microsoft/deberta-base", num_labels = 2)
model_bloom = BloomForSequenceClassification.from_pretrained("bigscience/bloom-560m", num_labels = 2)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at t5-small and are newly initialized: ['classification_head.out_proj.weight', 'classification_head.out_proj.bias', 'classification_head.dense.weight', 'classification_head.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


pytorch_model.bin:   0%|          | 0.00/559M [00:00<?, ?B/s]

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.weight', 'classifier.bias', 'pooler.dense.weight', 'pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of BloomForSequenceClassification were not initialized from the model checkpoint at bigscience/bloom-560m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
#odabir optimizatora
from torch.optim import AdamW
optimizer_t5_small = AdamW(model_t5_small.parameters(), lr = 5e-5)
optimizer_deberta = AdamW(model_deberta.parameters(), lr = 5e-5)
optimizer_bloom = AdamW(model_bloom.parameters(), lr = 5e-5)

In [11]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_deberta.to(device)

DebertaForSequenceClassification(
  (deberta): DebertaModel(
    (embeddings): DebertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=0)
      (LayerNorm): DebertaLayerNorm()
      (dropout): StableDropout()
    )
    (encoder): DebertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaLayer(
          (attention): DebertaAttention(
            (self): DisentangledSelfAttention(
              (in_proj): Linear(in_features=768, out_features=2304, bias=False)
              (pos_dropout): StableDropout()
              (pos_proj): Linear(in_features=768, out_features=768, bias=False)
              (pos_q_proj): Linear(in_features=768, out_features=768, bias=True)
              (dropout): StableDropout()
            )
            (output): DebertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): DebertaLayerNorm()
              (dropout): StableDropout()
            )
          )
          (

In [12]:
from transformers import get_scheduler
def training_params(num_of_epochs, optimizer):
  num_epochs = num_of_epochs
  num_training_steps = num_epochs * len(train_dataloader)
  lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
  )
  return num_training_steps, lr_scheduler

In [13]:
#podesavanja memorije zbog alokacije memorije prilikom treniranja koriscenjem PyTorch-a
import os
import gc
def mem_cleanup():
  os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:1024"
  torch.cuda.empty_cache()
  gc.collect()
mem_cleanup()

In [14]:
from huggingface_hub import notebook_login
notebook_login()

In [15]:
from tqdm.auto import tqdm
num_epochs = 5
num_training_steps, lr_scheduler = training_params(5, optimizer_deberta)
progress_bar = tqdm(range(num_training_steps))
model_deberta.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model_deberta(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer_deberta.step()
        lr_scheduler.step()
        optimizer_deberta.zero_grad()
        progress_bar.update(1)

  0%|          | 0/15625 [00:00<?, ?it/s]

In [16]:
model_deberta.push_to_hub("deberta_imdb")

model.safetensors:   0%|          | 0.00/557M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Minaaaa/deberta_imdb/commit/2d72f1b76ee6e962ee308e712a382d5b4be55af8', commit_message='Upload DebertaForSequenceClassification', commit_description='', oid='2d72f1b76ee6e962ee308e712a382d5b4be55af8', pr_url=None, pr_revision=None, pr_num=None)

In [17]:
import evaluate

metrics = evaluate.combine(["accuracy", "precision", "recall", "f1"])
model_deberta.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model_deberta(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metrics.add_batch(predictions=predictions, references=batch["labels"])

metrics.compute()

{'accuracy': 0.9312,
 'precision': 0.926829268292683,
 'recall': 0.9371519490851233,
 'f1': 0.9319620253164558}